We start importing the libraries we need.

In [2]:

from datetime import datetime, timedelta

import ipyplot
from dateutil.relativedelta import relativedelta
from IPython.display import Image
from pyinaturalist import (
    ICONIC_TAXA,
    Observation,
    TaxonCount,
    UserCount,
    enable_logging,
    get_observation_histogram,
    get_observation_identifiers,
    get_observation_observers,
    get_observation_species_counts,
    get_observations,
    pprint,
)
from rich import print

enable_logging()

In [3]:
import folium
import pandas as pd

In [42]:
def popup_html(df,row):
    name=df['taxon'].iloc[row] 
    place=df['place'].iloc[row] 
    url=df['url'].iloc[row]
    
    if df['EFG'].iloc[row] is None:
        html = """<!DOCTYPE html>
<html>
<head>
<h4 style="margin-bottom:10"; width="200px">{taxon}</h4>
</head>
<a href='{url}' target='inat'>iNat page</a>
    <table style="height: 126px; width: 200px;">
<tbody>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Place</span></td>
<td style="width: 60%;background-color: {color2};">{place}</td>
</tr>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Taxon</span></td>
<td style="width: 60%;background-color: {color2};">{taxon}</td>
</tr>
</tbody>
</table>
<span>{message}</span>
</html>
"""
        if df['has_context'].iloc[row] is None:
            message='This observation needs to be reviewed'
        elif df['has_context'].iloc[row] is True:
            message='This observation has been reviewed but some information is missing'
        else:
            message='This observation has been reviewed but does not have enough context to assign a Ecosystem Functional Group'
        
        output=html.format(taxon=name,place=place,url=url,EFG=None,comment=None,
                           color1="#a9a7bd",color2="#d2d0d3",message=message)

    else:
        efg=df['EFG'].iloc[row]
        comment=df['comment'].iloc[row]
    
        html = """<!DOCTYPE html>
<html>
<head>
<h4 style="margin-bottom:10"; width="200px">{taxon}</h4>
</head>
<a href='{url}' target='inat'>iNat page</a>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Place</span></td>
<td style="width: 60%;background-color: {color2};">{place}</td>
</tr>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Taxon</span></td>
<td style="width: 60%;background-color: {color2};">{taxon}</td>
</tr>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">EFG</span></td>
<td style="width: 60%;background-color: {color2};">{EFG}</td>
</tr>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Comment</span></td>
<td style="width: 60%;background-color: {color2};">{comment}</td>
</tr>
</tbody>
</table>
</html>
"""
        output=html.format(taxon=name,place=place,url=url,EFG=efg,comment=comment,color1="#19a7bd",color2="#f2f0d3")
        
    return output

In [43]:

tids = [48039, # Lobelias
        68790, # Gorillas
        341182 # Dendrosenecio
       ]
#tids = 341182
response=get_observations(place_id=7142, taxon_id=tids, page='all')
qry_observations = Observation.from_json_list(response)

[04-20 14:40:07] INFO     Request:                                                                   ]8;id=667057;file:///home/jferrer/proyectos/venv/ee/lib/python3.10/site-packages/pyinaturalist/session.py\session.py]8;;\:]8;id=127512;file:///home/jferrer/proyectos/venv/ee/lib/python3.10/site-packages/pyinaturalist/session.py#250\250]8;;\
                          GET                                                                                      
                          https://api.inaturalist.org/v1/observations?taxon_id=48039%2C68790%2C34118               
                          2&place_id=7142&only_id=false&per_page=200&order_by=id&order=asc                         
                          User-Agent: python-requests/2.28.2 pyinaturalist/0.18.0                                  
                          Accept-Encoding: gzip, deflate                                                           
                          Accept: application/json                                                                 
                          Connection: keep-alive                                                                   
                                                                                                                   

[04-20 14:40:13] INFO     This query will fetch 224 results in 2 requests. Estimated total request ]8;id=659570;file:///home/jferrer/proyectos/venv/ee/lib/python3.10/site-packages/pyinaturalist/paginator.py\paginator.py]8;;\:]8;id=333163;file:///home/jferrer/proyectos/venv/ee/lib/python3.10/site-packages/pyinaturalist/paginator.py#198\198]8;;\
                          time: 1 seconds                                                                          

                 INFO     Request:                                                                   ]8;id=15074;file:///home/jferrer/proyectos/venv/ee/lib/python3.10/site-packages/pyinaturalist/session.py\session.py]8;;\:]8;id=204400;file:///home/jferrer/proyectos/venv/ee/lib/python3.10/site-packages/pyinaturalist/session.py#250\250]8;;\
                          GET                                                                                      
                          https://api.inaturalist.org/v1/observations?taxon_id=48039%2C68790%2C34118               
                          2&place_id=7142&only_id=false&id_above=130761538&per_page=200&order_by=id&               
                          order=asc                                                                                
                          User-Agent: python-requests/2.28.2 pyinaturalist/0.18.0                                  
                          Accept-Encoding: gzip, deflate                                                           
                          Accept: application/json                                                                 
                          Connection: keep-alive                                                                   
                                                                                                                   

In [44]:
source = pd.DataFrame(
    [
        {
            'latitude': o.location[0],
            'longitude': o.location[1],
            'id':o.id,
            'url':o.uri,
            'place':o.place_guess,
            'taxon':o.species_guess,
            'iconic_taxon': o.taxon.iconic_taxon_name,
            'has_context':None,
            'reviewer':None,
            'EFG':None,
            'confidence':None,
            'comment':None,
        }
        for o in qry_observations
        if o.location
    ]
)

In [45]:
source.shape

(224, 12)

In [46]:
source.tail()

,latitude,longitude,id,url,place,taxon,iconic_taxon,has_context,reviewer,EFG,confidence,comment
219,-2.477868,29.226256,143936748,https://www.inaturalist.org/observations/14393...,"Iburengerazuba, RW",None,Plantae,None,None,None,None,None
220,-1.437719,29.795246,144164246,https://www.inaturalist.org/observations/14416...,Rwanda,Mountain Gorilla,Mammalia,None,None,None,None,None
221,-2.462299,29.246248,144463557,https://www.inaturalist.org/observations/14446...,"Nyamasheke, RW-OU, RW",Lobelia petiolata,Plantae,None,None,None,None,None
222,-2.440879,29.252687,144463734,https://www.inaturalist.org/observations/14446...,"Nyamasheke, RW-OU, RW",Lobelia giberroa,Plantae,None,None,None,None,None
223,-1.508119,29.419630,150942853,https://www.inaturalist.org/observations/15094...,Rwanda,Gorilla beringei beringei,Mammalia,None,None,None,None,None


In [47]:
 # initialize the map and store it in a m object
m = folium.Map(location = [-2, 29.5],
               zoom_start = 8)

In [48]:
locations = source[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [49]:
for point in range(0, len(locationlist)):
    context=source['has_context'].iloc[point]
    if context is None:
        color = 'gray'
    elif context is True:
        color = 'darkblue'
    else:
        color = 'lightred'
    folium.Marker(locationlist[point], icon=folium.Icon(color=color), popup=popup_html(source,point)).add_to(m)

In [50]:
m

In [ ]:
def evaluateContext(rid,context=None,validcoords=None):
    if context is None:
        input(...)

In [15]:
len(locationlist)


14

Another example for plants from the Grazing guidelines for Gayini.


In [30]:
pids = [49025]
tids = [502254, # Eucalyptus largiflorens # Black Box
        122825, # Acacia victoriae # Bramble Wattle
        52795, # Hordeum leporinum
        934705, # Maireana pyramidata
        291217, # Acacia salicina
        288066# Lachagrostis filiformis
    ]

response=get_observations(place_id=pids, taxon_id=tids, page='all')
qry_observations = Observation.from_json_list(response)

[04-20 14:35:25] INFO     Request:                                                                   ]8;id=206043;file:///home/jferrer/proyectos/venv/ee/lib/python3.10/site-packages/pyinaturalist/session.py\session.py]8;;\:]8;id=958290;file:///home/jferrer/proyectos/venv/ee/lib/python3.10/site-packages/pyinaturalist/session.py#250\250]8;;\
                          GET                                                                                      
                          https://api.inaturalist.org/v1/observations?taxon_id=502254%2C122825%2C527               
                          95%2C934705%2C291217%2C288066&place_id=49025&only_id=false&per_page=200&or               
                          der_by=id&order=asc                                                                      
                          User-Agent: python-requests/2.28.2 pyinaturalist/0.18.0                                  
                          Accept-Encoding: gzip, deflate                                                           
                          Accept: application/json                                                                 
                          Connection: keep-alive                                                                   
                                                                                                                   

In [31]:
source = pd.DataFrame(
    [
        {
            'latitude': o.location[0],
            'longitude': o.location[1],
            'id':o.id,
            'url':o.uri,
            'place':o.place_guess,
            'taxon':o.species_guess,
            'iconic_taxon': o.taxon.iconic_taxon_name,
            'has_context':None,
            'reviewer':None,
            'EFG':None,
            'confidence':None,
            'comment':None,
        }
        for o in qry_observations
        if o.location
    ]
)

In [32]:
source.tail()

,latitude,longitude,id,url,place,taxon,iconic_taxon,has_context,reviewer,EFG,confidence,comment
0,-34.430124,144.842637,47704600,https://www.inaturalist.org/observations/47704600,"Hay, AU-NS, AU",None,Plantae,None,None,None,None,None
1,-34.516033,144.843674,47704610,https://www.inaturalist.org/observations/47704610,"Hay, AU-NS, AU",None,Plantae,None,None,None,None,None
2,-34.239554,144.376457,131295113,https://www.inaturalist.org/observations/13129...,"Maude NSW 2711, Australia",Black Box,Plantae,None,None,None,None,None
3,-34.511721,144.834677,143704027,https://www.inaturalist.org/observations/14370...,"Hay NSW 2711, Australia",Black Box,Plantae,None,None,None,None,None


In [38]:
 # initialize the map and store it in a m object
m = folium.Map(location = [-34, 144],
               zoom_start = 8)
locations = source[['latitude', 'longitude']]
locationlist = locations.values.tolist()
for point in range(0, len(locationlist)):
    context=source['has_context'].iloc[point]
    if context is None:
        color = 'gray'
    elif context is True:
        color = 'darkblue'
    else:
        color = 'lightred'
    folium.Marker(locationlist[point], icon=folium.Icon(color=color), popup=popup_html(source,point)).add_to(m)

In [39]:
m